In [16]:
import pandas as pd

## GHP - podejście I

define the dataframe:

In [17]:
ilosc_tygod = 10

labels = ['przewidywany popyt','produkcja','dostepne']

przew_popyt = labels[0]
prod = labels[1]
dost = labels[2]

empt = [0 for x in range(ilosc_tygod)]

zad1 = {
    przew_popyt:[5,5,5, 5,5,5, 15,15,15, 15],
    prod:empt,
    dost:empt
}
zad2 = {
    przew_popyt:[0,0,0,0,20,0,40,0,0,0],
    prod:empt,
    dost:empt
}

ghp = pd.DataFrame(data=zad2, columns=labels, index=[x for x in range(1,ilosc_tygod+1)], dtype="int64")
ghp

,przewidywany popyt,produkcja,dostepne
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
5,20,0,0
6,0,0,0
7,40,0,0
8,0,0,0
9,0,0,0
10,0,0,0


define initial state: (na stanie, produkcja) 

In [18]:
na_stanie = 2
produkcja = 30


calculate GHP:

In [19]:

for i in range(1, ilosc_tygod+1):
    if i == 1: 
        dostepne_bez_produkcji = na_stanie - ghp.loc[i,przew_popyt]
    else: 
        dostepne_bez_produkcji = ghp.loc[i-1,dost] - ghp.loc[i,przew_popyt]
    
    if dostepne_bez_produkcji <= 0: 
        ghp.loc[i,dost] = dostepne_bez_produkcji + produkcja
        ghp.loc[i,prod] = produkcja
    else:
        ghp.loc[i,dost] = dostepne_bez_produkcji 
        

In [20]:
ghp.transpose()

,1,2,3,4,5,6,7,8,9,10
przewidywany popyt,0,0,0,0,20,0,40,0,0,0
produkcja,0,0,0,0,30,0,30,0,0,0
dostepne,2,2,2,2,12,12,2,2,2,2


## GHP - podejście II

In [21]:
# demand_input = input("Podaj tygodnie i odpowiadające im popyty (np. '5:12,7:8'): ")
demand_input = '5:20,7:40'
# production_input = input("Podaj tygodnie i odpowiadające im sztuki do wyprodukowania (np. '5:12,7:8'): ")
production_input = '5:18,7:40'
# available_input = input("Podaj obecnie dostępne sztuki towaru: ")
available_input = '2'

In [22]:
def calculate_production(production_input):
    production = [0 for x in range(10)]
    for pair in production_input.split(","):
        week, amount = pair.split(":")
        week = int(week.strip())
        amount = int(amount.strip())
        production[week-1] = amount
    return production

In [23]:
def calculate_demand(demand_input):
    demand = [0 for x in range(10)]
    for pair in demand_input.split(","):
        week, amount = pair.split(":")
        week = int(week.strip())
        amount = int(amount.strip())
        demand[week-1] = amount
    return demand

In [24]:
def calculate_available(available_input):
    available = [0 for x in range(10)]
    demand = calculate_demand(demand_input)
    production = calculate_production(production_input)
    available[0] = available_input
    for i in range(len(demand)):
        if demand[i] == 0: 
            available[i] = int(available_input)
        else:
            break
    for i in range(1, len(available)):
        if demand[i] != 0:
            available[i] = production[i] - demand[i] + available[i-1]
        else:
            try:
                available[i] = available[i-1]
            except:
                pass
    return available
        

In [25]:
import pandas as pd

In [26]:
ghp = {
    'tydzień' : [1,2,3,4,5,6,7,8,9,10],
    'przewidywany popyt' : calculate_demand(demand_input),
    'produkcja' : calculate_production(production_input),
    'dostepne' : calculate_available(available_input)
}

In [27]:
df = pd.DataFrame(ghp).transpose()
df

,0,1,2,3,4,5,6,7,8,9
tydzień,1,2,3,4,5,6,7,8,9,10
przewidywany popyt,0,0,0,0,20,0,40,0,0,0
produkcja,0,0,0,0,18,0,40,0,0,0
dostepne,2,2,2,2,0,0,0,0,0,0
